# 🚀 Deploy Pipeline to Vertex AI - Complete Guide

මෙම notebook එක භාවිතා කරලා **complete pipeline** (vectorizer + model) Vertex AI එකට deploy කරන්න පුළුවන්.

**මෙතනදී වෙන්නේ:**

1. Pipeline එක GCS Bucket එකට upload කිරීම
2. Vertex AI Model Registry එකට register කිරීම
3. Endpoint එකක් create කරලා deploy කිරීම
4. Raw text යවලා predictions ගැනීම (vectorizer නැතුව!)


## Prerequisites (පළමුව මේවා කරන්න)

1. ✅ Pipeline එක train කරලා `model.joblib` විදියට save කරලා තියෙන්න ඕනේ
2. ✅ Google Cloud Project එකක් හදලා billing enable කරන්න
3. ✅ Vertex AI API සහ Cloud Storage API enable කරන්න
4. ✅ Authentication කරගන්න: `gcloud auth application-default login`


In [ ]:
# Import Required Libraries
from google.cloud import aiplatform, storage
import os

print("Libraries imported successfully!")

## Step 1: Configuration

ඔබගේ Project details fill කරන්න:


In [ ]:
# --- Configuration ---
PROJECT_ID = "project-968ac48e-8511-4620-9fe"  # Your Project ID
REGION = "asia-southeast1"  # Singapore region
BUCKET_NAME = "your-bucket-name"  # GCS bucket name (WITHOUT gs:// prefix)

# Model details
MODEL_DISPLAY_NAME = "sentiment-pipeline-v2"  # New name for pipeline deployment
ENDPOINT_DISPLAY_NAME = "Pipeline_Deployment"  # Your existing endpoint name

# Local file path
LOCAL_MODEL_PATH = "model.joblib"  # The pipeline file

print(f"✅ Project: {PROJECT_ID}")
print(f"✅ Region: {REGION}")
print(f"✅ Bucket: gs://{BUCKET_NAME}")
print(f"✅ Model file: {LOCAL_MODEL_PATH}")

## Step 2: Create GCS Bucket (if needed)

Bucket එකක් නැත්නම් මේක run කරන්න:


In [ ]:
# Create GCS Bucket (uncomment if you need to create one)
# storage_client = storage.Client(project=PROJECT_ID)

# try:
#     bucket = storage_client.create_bucket(BUCKET_NAME, location=REGION)
#     print(f"✅ Bucket {bucket.name} created in {REGION}")
# except Exception as e:
#     if "already exists" in str(e):
#         print(f"✅ Bucket {BUCKET_NAME} already exists")
#     else:
#         print(f"❌ Error: {e}")

## Step 3: Upload Pipeline to GCS

Pipeline එක (model.joblib) bucket එකට upload කරන්න:


In [ ]:
# Upload model.joblib to GCS bucket
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

# Create a folder structure: model/model.joblib
blob = bucket.blob("model/model.joblib")

# Upload the file
print("📤 Uploading pipeline to GCS...")
blob.upload_from_filename(LOCAL_MODEL_PATH)

print(f"✅ Pipeline uploaded to: gs://{BUCKET_NAME}/model/model.joblib")

# This is the artifact URI you'll use for deployment
ARTIFACT_URI = f"gs://{BUCKET_NAME}/model"
print(f"📍 Artifact URI: {ARTIFACT_URI}")

## Step 4: Initialize Vertex AI


In [ ]:
# Initialize Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION)
print("✅ Vertex AI initialized")

## Step 5: Upload Model to Vertex AI Registry

Pipeline එක Vertex AI Model Registry එකට register කරන්න:

**වැදගත්:** Scikit-learn version එක හරි container එක තෝරන්න!


In [ ]:
# Upload model to Vertex AI Model Registry
print("📦 Uploading model to Vertex AI Registry...")
print("⏳ This may take 5-10 minutes...")

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",
    # For different sklearn versions, use:
    # sklearn 1.2: "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest"
    # sklearn 1.3: "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"
)

print(f"✅ Model uploaded successfully!")
print(f"Model resource name: {model.resource_name}")

## Step 6: Deploy to Endpoint

දැනට තියෙන endpoint එක use කරන්න හෝ අලුත් එකක් හදන්න:


In [ ]:
# Option A: Use existing endpoint
print("🔍 Looking for existing endpoint...")
endpoints = aiplatform.Endpoint.list(filter=f'display_name="{ENDPOINT_DISPLAY_NAME}"')

if endpoints:
    endpoint = endpoints[0]
    print(f"✅ Found existing endpoint: {endpoint.display_name}")
    print(f"Resource name: {endpoint.resource_name}")
else:
    # Option B: Create new endpoint if not found
    print("Creating new endpoint...")
    endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)
    print(f"✅ Endpoint created: {endpoint.display_name}")

print(f"\n📍 Endpoint resource: {endpoint.resource_name}")

In [ ]:
# Deploy model to endpoint
print("🚀 Deploying model to endpoint...")
print("⏳ This will take 10-15 minutes...")

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name="pipeline-v2",
    machine_type="n1-standard-2",  # You can use n1-standard-4 for more power
    min_replica_count=1,
    max_replica_count=1,
    traffic_percentage=100,  # Route 100% traffic to this version
)

print("\n🎉 Deployment completed successfully!")
print(f"✅ Endpoint: {endpoint.display_name}")
print(f"✅ Resource: {endpoint.resource_name}")
print("\n💡 Now you can send RAW TEXT directly to get predictions!")

## Step 7: Test with Raw Text

දැන් vectorizer නැතුව කෙලින්ම raw text යවන්න පුළුවන්!


In [ ]:
# Test prediction with raw text
new_review = "I recently purchased this dress and I have to say, it exceeded my expectations!"

print(f"Testing with review: '{new_review}'")
print("\n...Calling endpoint...")

# Send raw text directly - no vectorizer needed!
response = endpoint.predict(instances=[new_review])

print("\n--- Prediction Result ---")
print(f"Prediction: {response.predictions[0]}")

if response.predictions[0] == 1:
    print("📊 Classification: POSITIVE review ✓")
else:
    print("📊 Classification: NEGATIVE review ✗")

## Step 8: Test with Multiple Reviews


In [ ]:
# Test with multiple reviews
test_reviews = [
    "I absolutely love this dress! The fit is perfect and the material is so soft.",
    "The shirt shrunk after one wash. Very disappointed with the quality.",
    "Amazing product! Highly recommend to everyone!",
    "Terrible quality. Waste of money."
]

print("Testing multiple reviews...\n")

for i, review in enumerate(test_reviews, 1):
    response = endpoint.predict(instances=[review])
    sentiment = "POSITIVE ✓" if response.predictions[0] == 1 else "NEGATIVE ✗"
    print(f"{i}. {review}")
    print(f"   → {sentiment}\n")

## 🎯 Summary

### විජයගිය!

දැන් ඔබට:

✅ Pipeline (vectorizer + model) Vertex AI එකට deploy කරලා තියෙනවා  
✅ Raw text කෙලින්ම endpoint එකට යවන්න පුළුවන්  
✅ Vectorizer එක locally load කරන්න අවශ්‍ය නැහැ  
✅ Production-ready deployment එකක් තියෙනවා

### Useful Commands:

```python
# List all models
models = aiplatform.Model.list()

# List all endpoints
endpoints = aiplatform.Endpoint.list()

# Undeploy old version
# endpoint.undeploy_all()

# Delete endpoint
# endpoint.delete()
```
